# Load libraries

In [1]:
### Load libraries 
import os
import numpy as np
import pandas as pd

from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

from ipywidgets import *
from IPython.display import clear_output, display
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

import qgrid

import helpers.research_helpers as app

CSV_SEPARATOR = ','     # '\t'

print('Notebook initialized.')

Notebook initialized.


# Load data

In [2]:
### Load membership list from '../s_drive_sampling/All Active Roster.csv' 

df_all_active_roster = app.load_csv(FLDR_SAMPLING + '/All Active Roster.csv')
print('Data loaded from ' + FLDR_SAMPLING + '/All Active Roster.csv')

NameError: name 'FLDR_SAMPLING' is not defined

# Filter by... membership_items

In [ ]:
### Please select those membership_items you would like to keep. 
# The default list (keep_records.KEEP_LIST_MEMBERSHIP_ITEMS) is already selected.

# Allow user's to select which membership_item groups, if any, they want to exclude
qgrid_sheet_to_keep = app.keep_grid_show_filter(df_all_active_roster, app.COL_MEMBERSHIP_ITEM, app.COL_MEMBER_ID, app.KEEP_LIST_MEMBERSHIP_ITEMS)
qgrid_sheet_to_keep

## Apply changes

In [ ]:
### Apply filtering 
#
#To just get the dataframe returned by qgrid_sheet_to_keep, just run the next 2 lines s
#df_filter_by = qgrid_sheet_to_keep.get_changed_df()
#df_filter_by.head(40)

df_filtered_list = app.keep_grid_apply_filter(qgrid_sheet_to_keep, app.COL_MEMBERSHIP_ITEM, df_all_active_roster)
print('Filtering applied.')

# Filter by... membership expiration date range

In [ ]:
### Please select those expiration groups you would like to keep. 
## The default list (keep_records.KEEP_LIST_EXPIRES_IN) is already selected.

### Calculate Expiration Range based off of EXPIRY_DATE and store in EXPIRY_RANGE_CALCULATED
df_filtered_list = app.assign_expiration_range(df_filtered_list)

# Allow user's to select which expiration groups, if any, they want to exclude
qgrid_sheet_to_keep = app.keep_grid_show_filter(df_filtered_list, app.COL_EXPIRY_RANGE_CALCULATED, app.COL_MEMBER_ID, app.KEEP_LIST_EXPIRES_IN)
qgrid_sheet_to_keep

## Apply changes

In [ ]:
### Apply filtering 
#
# To just get the dataframe returned by qgrid_sheet_to_keep, just run the next 2 lines s
#df_filter_by = qgrid_sheet_to_keep.get_changed_df()
#df_filter_by.head(40)

df_filtered_list = app.keep_grid_apply_filter(qgrid_sheet_to_keep, app.COL_EXPIRY_RANGE_CALCULATED, df_filtered_list)
print('Filtering applied.')

# Calculate... manager/worker

In [ ]:
### Calculate Title based off of JOB_TITLE, BOSS_TITLE, and NUM_OVERSEEN and store in TITLE_CALCULATED 

df_filtered_list = app.assign_title(df_filtered_list)

df_test = df_filtered_list.filter([app.COL_JOB_TITLE, app.COL_BOSS_TITLE, app.COL_NUM_OVERSEEN, app.COL_TITLE_CALCULATED])
qgrid_widget = qgrid.show_grid(df_test, show_toolbar=False)
qgrid_widget

print('The grid below is meant as informational purposes only.')

# Calculate... pub/private sector

In [ ]:
### Calculate Pub/Private Sector based off of ORG_TYPE, INDUSTRY, and EMAIL_DOMAIN and store in PUB_PRIVATE_CALCULATED 

df_filtered_list = app.assign_pub_private_sector(df_filtered_list)

df_test = df_filtered_list.filter([app.COL_ORG_TYPE, app.COL_INDUSTRY, app.COL_EMAIL_DOMAIN, app.COL_PUB_PRIVATE_CALCULATED])
qgrid_widget = qgrid.show_grid(df_test, show_toolbar=False)
qgrid_widget

print('The grid below is meant as informational purposes only.')

# Filter by... job_title

In [ ]:
### Please select those job titles you would like to keep. 

qgrid_sheet_to_keep = app.keep_grid_show_filter(df_filtered_list, app.COL_JOB_TITLE, app.COL_MEMBER_ID, app.KEEP_LIST_JOB_TITLES)
qgrid_sheet_to_keep

## Apply changes

In [ ]:
### Apply filtering 

# To just get the dataframe returned by qgrid_sheet_to_keep, just run the next 2 lines s
#df_filter_by = qgrid_sheet_to_keep.get_changed_df()
#df_filter_by.head(40)

df_filtered_list = app.keep_grid_apply_filter(qgrid_sheet_to_keep, app.COL_JOB_TITLE, df_filtered_list)
print('Filtering applied.')

# Filter by... survey recency

## Apply changes

# Save output

## Get project/directory name

In [ ]:
### Add code to save project and data files to a directory of the user's choosing

project_name_lbl = widgets.Label('Project Name: ')
project_name_txt = widgets.Text()
display(project_name_lbl)
display(project_name_txt)

In [ ]:
project_name_results_lbl = widgets.Label()
project_name_results_lbl.value = 'Your data will be saved to ../s_drive_sampling/' + project_name_txt.value + '/'
display(project_name_results_lbl)

In [ ]:
### Create folder in ../s_drive_sampling/<project_name> 



## Save project files